In [1]:
#Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from resources import gkey

gmaps.configure(api_key = gkey)

In [2]:
#Extract and explore data from cities data of previous exercise.
cities_data_df = pd.read_csv("output_data/cities.csv")
cities_data_df.head()

,City,Cloudiness,Contry,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ushuaia,75,AR,1585709569,81,-54.80,-68.30,46.4,10.29
1,Hong Kong,40,HK,1585709939,88,22.29,114.16,70.0,12.75
2,Niamey,14,NE,1585709941,13,13.51,2.11,86.0,3.36
3,Hithadhoo,93,MV,1585709942,66,-0.60,73.08,83.3,9.15
4,Vardø,75,NO,1585709942,85,70.37,31.11,23.0,21.92


In [3]:
#Show humidity heatmap from cities data.
fig = gmaps.figure()
locations = cities_data_df[["Lat", "Lng"]]
weights = cities_data_df["Humidity"]
fig.add_layer(gmaps.heatmap_layer(locations, weights = weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Define vacation candidates based on ideal weather conditions:

# Max temperature between 70 and 80 °F. Ideal warmth.
candidates_df = cities_data_df.loc[(cities_data_df["Max Temp"] < 80) & (cities_data_df["Max Temp"] > 70) ,:]

# No cloudiness. We want sun!
candidates_df = candidates_df.loc[(cities_data_df["Cloudiness"] == 0), :]

# Wind speed less than 10 mph. Not too windy!
candidates_df = candidates_df.loc[(cities_data_df["Wind Speed"] < 5), :]


In [5]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
locations = []
hotel_names = []
cities = []
countries = []
infoBoxes = []



for index, row in candidates_df.iterrows():
    
    params = {"location": str(row["Lat"]) + "," + str(row["Lng"]),
                "radius": 5000,
                "type": "hotel",
                "key": gkey}
                
    request = requests.get(url, params = params)
    
    response = request.json()
        
    try:
        location = (response["results"][0]["geometry"]["location"]["lat"],response["results"][0]["geometry"]["location"]["lng"])
        locations.append(location)
        hotel_names.append(response["results"][0]["name"])
        cities.append(row["City"])
        countries.append(row["Contry"])
    
        infoBoxes.append("Hotel name: " + response["results"][0]["name"] + ", City: " + row["City"] + ", Country: " + row["Contry"])

    except:
        print("No hotel found! Skipping...")


markers = gmaps.marker_layer(locations, info_box_content = infoBoxes)
fig.add_layer(markers)
display(fig)

Figure(layout=FigureLayout(height='420px'))